# Important Libraries 

In [48]:
import pandas as pd                        
import numpy as np
import csv
import openpyxl as xls
import re    
import io
import os
from datetime import date

In [41]:
os.chdir('J:/DataScience/DataScience&Analytics/ODP development/Border Crossing/Data')


# Read in downloaded data, add a port name column and rename columns. Drop empty rows 

In [42]:
os.chdir('J:/DataScience/DataScience&Analytics/ODP development/Border Crossing/Data')
port_Trade_Data = pd.read_csv("dot3_ytd_0323.csv")
port_Trade_Data.index = port_Trade_Data.index + 1
port_Trade_Data=port_Trade_Data.rename(columns = {'DEPE': 'Port Code','FREIGHT_CHARGES': 'Freight Charge', 'COMMODITY2': 'Commodity', 'DISAGMOT': 'Mode of Transportation', 'TRDTYPE': 'Trade Type','COUNTRY': 'Country','VALUE': 'Value','SHIPWT': 'Shipment Weight','DF': 'Production Location','CONTCODE': 'Container','MONTH': 'Month','Year': 'Year', })
port_Trade_Data=port_Trade_Data.rename(columns = {'YEAR': 'Year'})
port_Trade_Data=port_Trade_Data.dropna(how='all')
port_Trade_Data['Port Name'] = port_Trade_Data.loc[:, 'Port Code']
port_Trade_Data['Year'] = port_Trade_Data['Year'].astype(str)
port_Trade_Data['Year']=port_Trade_Data['Year'].str.replace(',','')
port_Trade_Data['Month&Year'] = port_Trade_Data['Month'].astype(str)+'/1/'+port_Trade_Data['Year']
port_Trade_Data['Year']='1/1/'+port_Trade_Data['Year']
port_Trade_Data['Month_Text'] = port_Trade_Data.loc[:, 'Month']
port_Trade_Data['Month_Text'] = port_Trade_Data['Month_Text'].replace({1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'})
port_Trade_Data

,Trade Type,Port Code,Commodity,Mode of Transportation,Country,Value,Shipment Weight,Freight Charge,Production Location,Container,Month,Year,Port Name,Month&Year,Month_Text
1,1,0101,29,6,1220,301680,0,74,2.0,X,1,1/1/2023,0101,1/1/2023,January
2,1,0101,73,5,1220,2842,0,43,1.0,X,1,1/1/2023,0101,1/1/2023,January
3,1,0101,73,5,1220,13423,0,132,2.0,X,1,1/1/2023,0101,1/1/2023,January
4,1,0101,73,6,1220,23851,0,707,1.0,X,1,1/1/2023,0101,1/1/2023,January
5,1,0101,85,5,1220,10677,0,253,1.0,X,1,1/1/2023,0101,1/1/2023,January
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52032,2,55XX,98,5,2010,13411,1128,8,NaN,0,3,1/1/2023,55XX,3/1/2023,March
52033,2,60XX,98,8,1220,12995,86,450,NaN,0,3,1/1/2023,60XX,3/1/2023,March
52034,2,60XX,98,8,2010,43000,7020,4505,NaN,0,3,1/1/2023,60XX,3/1/2023,March
52035,2,70XX,99,8,1220,246890556,0,0,NaN,0,3,1/1/2023,70XX,3/1/2023,March


In [43]:
mappingName = {'2504':'San Ysidro','2505':'Tecate','2507':'Calexico-East','2502':'Andrade','2503':'Calexico','2506':'Otay Mesa Station'}
port_Trade_Data['Port Name'] = port_Trade_Data['Port Name'].map(mappingName) #delete all values in Port Name that arent in San Diego County
port_Trade_Data['Production Location'] = port_Trade_Data['Production Location'].replace({1.0:'Domestically Produced Goods', 2.0:'Foreign Produced Goods'})
port_Trade_Data['Country'] = port_Trade_Data['Country'].replace({1220:'Canada',1220:'Canada', 2010: 'Mexico'})
port_Trade_Data['Container'] = port_Trade_Data['Container'].replace({1:'Containerized',0:'Not Containerized', 'X': 'Unknown','': 'Non Containerized', '1':'Containerized','0':'Not Containerized'})
port_Trade_Data['Trade Type'] = port_Trade_Data['Trade Type'].replace({2:'Import',1:'Export'})
port_Trade_Data['Commodity'] = port_Trade_Data['Commodity'].map({1: 'Live animals', 2: 'Meat and edible meat offal', 3: 'Fish and crustaceans mollusks and other aquatic invertebrates', 4: 'Dairy produce Birds eggs Natural honey Edible products of animal origin not elsewhere specified or included', 5: 'Products of animal origin not elsewhere specified or included', 6: 'Live trees and other plants Bulbs roots and the like Cut flowers and ornamental foliage', 7: 'Edible vegetables and certain roots and tubers', 8: 'Edible fruit and nuts Peel of citrus fruit or melons', 9: 'Coffee tea mate and spices', 10: 'Cereals', 11: 'Products of the milling industry Malt Starches inulin Wheat gluten', 12: 'Oil seeds and oleaginous fruits Miscellaneous grains Seeds and fruit Industrial or medicinal plants Straw and fodder', 13: 'Lac Gums Resins and other vegetable saps and extract', 14: 'Vegetable plaiting materials Vegetable products not elsewhere specified or included', 15: 'Animal or vegetable fats and oils and their cleavage products Prepared edible fats Animal or vegetable waxes', 16: 'Preparations of meat of fish or of crustaceans mollusks or other aquatic invertebrates', 17: 'Sugars and sugar confectionery', 18: 'Cocoa and cocoa preparations', 19: 'Preparations of cereals flour starch or milk Bakers wares', 20: 'Preparations of vegetables fruit nuts or other parts of plants', 21: 'Miscellaneous edible preparations', 22: 'Beverages spirits and vinegar', 23: 'Residues and waste from the food industries Prepared animal feed', 24: 'Tobacco and manufactured tobacco substitutes', 25: 'Salt Sulfur Earths and stone Plastering materials lime and cement', 26: 'Ores slag and ash', 27: 'Mineral fuels mineral oils and products of their distillation Bituminous substances Mineral waxes', 28: 'Inorganic chemicals Organic or inorganic compounds of precious metals of rare earth metals of radioactive elements or of isotopes', 29: 'Organic chemicals', 30: 'Pharmaceutical products', 31: 'Fertilizers', 32: 'Tanning or dyeing extracts Tannins and their derivatives Dyes pigments and other coloring matter Paints and varnishes Putty and other mastics Inks', 33: 'Essential oils and resinoids Perfumery cosmetic or toilet preparations', 34: 'Soap organic surface active agents washing preparations lubricating preparations artificial waxes prepared waxes polishing or scouring preparations candles and similar articles modeling pastes dental waxes and dental preparations with a basis of plaster', 35: 'Albuminoidal substances Modified starches Glues Enzymes', 36: 'Explosives Pyrotechnic products Matches Pyrophoric alloys Certain combustible preparations', 37: 'Photographic or cinematographic goods', 38: 'Miscellaneous chemical products', 39: 'Plastics and articles thereof', 40: 'Rubber and articles thereof', 41: 'Raw hides and skins other than furskins and leather', 42: 'Articles of leather Saddlery and harness Travel goods handbags and similar containers Articles of animal gut other than silkworm gut', 43: 'Furskins and artificial fur Manufactures thereof', 44: 'Wood and articles of wood Wood charcoal', 45: 'Cork and articles of cork', 46: 'Manufactures of straw of esparto or of other plaiting materials Basketware and wickerwork', 47: 'Pulp of wood or of other fibrous cellulosic material Waste and scrap of paper or paperboard', 48: 'Paper and paperboard Articles of paper pulp of paper or of paperboard', 49: 'Printed books newspapers pictures and other products of the printing industry Manuscripts typescripts and plans', 50: 'Silk', 51: 'Wool fine or coarse animal hair Horsehair yarn and woven fabric', 52: 'Cotton', 53: 'Other vegetable textile fibers Paper yarn and woven fabrics of paper yarn', 54: 'Man made filaments', 55: 'Man made staple fibers', 56: 'Wadding felt and nonwovens Special yarns Twine cordage ropes and cables and articles thereof', 57: 'Carpets and other textile floor coverings', 58: 'Special woven fabrics Tuffed textile fabrics Lace Tapestries Trimmings Embroidery', 59: 'Impregnated coated covered or laminated textile fabrics Textile articles of a kind suitable for industrial use', 60: 'Knitted or crocheted fabrics', 61: 'Articles of apparel and clothing accessories knitted or crocheted', 62: 'Articles of apparel and clothing accessories not knitted or crocheted', 63: 'Other made up textile articles Needle craft sets Worn clothing and worn textile articles rags', 64: 'Footwear gaiters and the like Parts of such articles', 65: 'Headgear and parts thereof', 66: 'Umbrellas sun umbrellas walking sticks seatsticks whips riding crops and parts thereof', 67: 'Prepared feathers and down and articles made of feathers or of down artificial flowers articles of human hair', 68: 'Articles of stone plaster cement asbestos mica or similar materials', 69: 'Ceramic products', 70: 'Glass and glassware', 71: 'Natural or cultured pearls precious or semiprevious stones precious metals metals clad with precious metal and articles thereof imitation jewelry coin', 72: 'Iron and steel', 73: 'Articles of iron or steel', 74: 'Copper and articles thereof', 75: 'Nickel and articles thereof', 76: 'Aluminum and articles thereof', 77: 'Reserved for possible future use', 78: 'Lead and articles thereof', 79: 'Zinc and articles thereof', 80: 'Tin and articles thereof', 81: 'Other base metals Cermets Articles thereof', 82: 'Tools implements cutlery spoons and forks of base metal Parts thereof of base metal', 83: 'Miscellaneous articles of base metal', 84: 'Nuclear reactors boilers machinery and mechanical appliances parts thereof', 85: 'Electrical machinery and equipment and parts thereof Sound recorders and reproducers television image and sound recorders and reproducers and parts and accessories of such articles', 86: 'Railway or tramway locomotives rolling stock and parts thereof railway or tramway track fixtures and fittings and parts thereof Mechanical including electromechanical traffic signaling equipment of all kinds', 87: 'Vehicles other than railway or tramway rolling stock and parts and accessories thereof', 88: 'Aircraft spacecraft and parts thereof', 89: 'Ships boats and floating structures', 90: 'Optical photographic cinematographic measuring checking precision medical or surgical instruments and apparatus Parts and accessories thereof', 91: 'Clocks and watches and parts thereof', 92: 'Musical instruments Parts and accessories of such articles', 93: 'Arms and ammunition Parts and accessories thereof', 94: 'Furniture Bedding mattress supports cushions and similar stuffed furnishings Lamps and lighting fittings not elsewhere specified or included Illuminated signs illuminated nameplates and the like Prefabricated buildings', 95: 'Toys games and sports equipment Parts and accessories thereof', 96: 'Miscellaneous manufactured articles', 97: 'Works of art collectors pieces and antiques', 98: 'Special classification provisions', 99: 'Imports only Temporary legislation Temporary modifications established pursuant to trade legislation Additional import restrictions established pursuant to Section of the Agricultural Adjustment Act as needed'})
port_Trade_Data['Mode of Transportation'] = port_Trade_Data['Mode of Transportation'].replace({1:'Vessel',3:'Air',4:'Mail (U.S. Postal Service)', 5:'Truck',6:'Rail',7:'Pipeline',8:'Other',9:'Foreign Trade Zones (FTZs)'})

port_Trade_Data

,Trade Type,Port Code,Commodity,Mode of Transportation,Country,Value,Shipment Weight,Freight Charge,Production Location,Container,Month,Year,Port Name,Month&Year,Month_Text
1,Export,0101,Organic chemicals,Rail,Canada,301680,0,74,Foreign Produced Goods,Unknown,1,1/1/2023,NaN,1/1/2023,January
2,Export,0101,Articles of iron or steel,Truck,Canada,2842,0,43,Domestically Produced Goods,Unknown,1,1/1/2023,NaN,1/1/2023,January
3,Export,0101,Articles of iron or steel,Truck,Canada,13423,0,132,Foreign Produced Goods,Unknown,1,1/1/2023,NaN,1/1/2023,January
4,Export,0101,Articles of iron or steel,Rail,Canada,23851,0,707,Domestically Produced Goods,Unknown,1,1/1/2023,NaN,1/1/2023,January
5,Export,0101,Electrical machinery and equipment and parts t...,Truck,Canada,10677,0,253,Domestically Produced Goods,Unknown,1,1/1/2023,NaN,1/1/2023,January
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52032,Import,55XX,Special classification provisions,Truck,Mexico,13411,1128,8,NaN,Not Containerized,3,1/1/2023,NaN,3/1/2023,March
52033,Import,60XX,Special classification provisions,Other,Canada,12995,86,450,NaN,Not Containerized,3,1/1/2023,NaN,3/1/2023,March
52034,Import,60XX,Special classification provisions,Other,Mexico,43000,7020,4505,NaN,Not Containerized,3,1/1/2023,NaN,3/1/2023,March
52035,Import,70XX,Imports only Temporary legislation Temporary m...,Other,Canada,246890556,0,0,NaN,Not Containerized,3,1/1/2023,NaN,3/1/2023,March


# Reading Previous Data

In [44]:
#Use this data set when initially uploading the data.If ran again, will create an additional 17k rows because the previous data will include the current month
#previous_Trade_data = pd.read_csv('Port_and_Commodity_TransBorder_-_January_2006_to_Latest_Available_Month.csv')

#Use this dataset when testing df. 
previous_Trade_data = pd.read_csv('Port_and_Commodity_TransBorder_-_January_2006_to_Latest_Available_Month.csv')

previous_Trade_data.index = previous_Trade_data.index + 1
previous_Trade_data=previous_Trade_data.rename(columns = {'DEPE': 'Port Code','FREIGHT_CHARGES': 'Freight Charge', 'COMMODITY2': 'Commodity', 'DISAGMOT': 'Mode of Transportation', 'TRDTYPE': 'Trade Type','COUNTRY': 'Country','VALUE': 'Value','SHIPWT': 'Shipment Weight','DF': 'Production Location','CONTCODE': 'Container','MONTH': 'Month','Year': 'Year', })
previous_Trade_data=previous_Trade_data.rename(columns = {'YEAR': 'Year'})
previous_Trade_data=previous_Trade_data.dropna(how='all')
previous_Trade_data

,Trade Type,Port Code,Commodity,Mode of Transportation,Country,Value,Shipment Weight,Freight Charge,Production Location,Container,Month,Year,Port Name,Month&Year,Month_Text
1,Export,0101,Products of animal origin not elsewhere specif...,Truck,Canada,27500,0,539,Foreign Produced Goods,Unknown,1,1/1/2022,NaN,1/1/2022,January
2,Export,0101,Oil seeds and oleaginous fruits Miscellaneous ...,Truck,Canada,8576,0,167,Domestically Produced Goods,Unknown,1,1/1/2022,NaN,1/1/2022,January
3,Export,0101,Preparations of cereals flour starch or milk B...,Truck,Canada,41359,0,810,Domestically Produced Goods,Unknown,1,1/1/2022,NaN,1/1/2022,January
4,Export,0101,Salt Sulfur Earths and stone Plastering materi...,Truck,Canada,14615,0,286,Domestically Produced Goods,Unknown,1,1/1/2022,NaN,1/1/2022,January
5,Export,0101,Organic chemicals,Truck,Canada,119860,0,2047,Domestically Produced Goods,Unknown,1,1/1/2022,NaN,1/1/2022,January
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3435770,Import,55XX,Special classification provisions,Truck,Canada,3132978,4396,2857,NaN,Not Containerized,12,1/1/2022,NaN,12/1/2022,December
3435771,Import,60XX,Ships boats and floating structures,Other,Canada,14944,680,150,NaN,Not Containerized,12,1/1/2022,NaN,12/1/2022,December
3435772,Import,60XX,Special classification provisions,Other,Canada,5000,17,450,NaN,Not Containerized,12,1/1/2022,NaN,12/1/2022,December
3435773,Import,70XX,Imports only Temporary legislation Temporary m...,Other,Canada,207760408,0,0,NaN,Not Containerized,12,1/1/2022,NaN,12/1/2022,December


In [45]:
#previous_Trade_data = previous_Trade_data[~(previous_Trade_data['Month&Year'].isin(['1/1/2022', '2/1/2022']))]
#previous_Trade_data

# Combine Previous data

In [46]:
port_Trade_Data = pd.concat([previous_Trade_data, port_Trade_Data])

port_Trade_Data.drop_duplicates()

#port_Trade_Data.reset_index(inplace = False)


,Trade Type,Port Code,Commodity,Mode of Transportation,Country,Value,Shipment Weight,Freight Charge,Production Location,Container,Month,Year,Port Name,Month&Year,Month_Text
1,Export,0101,Products of animal origin not elsewhere specif...,Truck,Canada,27500,0,539,Foreign Produced Goods,Unknown,1,1/1/2022,NaN,1/1/2022,January
2,Export,0101,Oil seeds and oleaginous fruits Miscellaneous ...,Truck,Canada,8576,0,167,Domestically Produced Goods,Unknown,1,1/1/2022,NaN,1/1/2022,January
3,Export,0101,Preparations of cereals flour starch or milk B...,Truck,Canada,41359,0,810,Domestically Produced Goods,Unknown,1,1/1/2022,NaN,1/1/2022,January
4,Export,0101,Salt Sulfur Earths and stone Plastering materi...,Truck,Canada,14615,0,286,Domestically Produced Goods,Unknown,1,1/1/2022,NaN,1/1/2022,January
5,Export,0101,Organic chemicals,Truck,Canada,119860,0,2047,Domestically Produced Goods,Unknown,1,1/1/2022,NaN,1/1/2022,January
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52032,Import,55XX,Special classification provisions,Truck,Mexico,13411,1128,8,NaN,Not Containerized,3,1/1/2023,NaN,3/1/2023,March
52033,Import,60XX,Special classification provisions,Other,Canada,12995,86,450,NaN,Not Containerized,3,1/1/2023,NaN,3/1/2023,March
52034,Import,60XX,Special classification provisions,Other,Mexico,43000,7020,4505,NaN,Not Containerized,3,1/1/2023,NaN,3/1/2023,March
52035,Import,70XX,Imports only Temporary legislation Temporary m...,Other,Canada,246890556,0,0,NaN,Not Containerized,3,1/1/2023,NaN,3/1/2023,March


# Dropping Duplicate Rows

In [47]:
port_Trade_Data.drop_duplicates()
port_Trade_Data['Month&Year'] = port_Trade_Data['Month&Year'].str.replace(r'\b(\d)/', r'0\1/', regex=True)

port_Trade_Data.to_csv('Port_and_Commodity_TransBorder_-_January_2006_to_Latest_Available_Month.csv', index=False)

port_Trade_Data

,Trade Type,Port Code,Commodity,Mode of Transportation,Country,Value,Shipment Weight,Freight Charge,Production Location,Container,Month,Year,Port Name,Month&Year,Month_Text
1,Export,0101,Products of animal origin not elsewhere specif...,Truck,Canada,27500,0,539,Foreign Produced Goods,Unknown,1,1/1/2022,NaN,01/01/2022,January
2,Export,0101,Oil seeds and oleaginous fruits Miscellaneous ...,Truck,Canada,8576,0,167,Domestically Produced Goods,Unknown,1,1/1/2022,NaN,01/01/2022,January
3,Export,0101,Preparations of cereals flour starch or milk B...,Truck,Canada,41359,0,810,Domestically Produced Goods,Unknown,1,1/1/2022,NaN,01/01/2022,January
4,Export,0101,Salt Sulfur Earths and stone Plastering materi...,Truck,Canada,14615,0,286,Domestically Produced Goods,Unknown,1,1/1/2022,NaN,01/01/2022,January
5,Export,0101,Organic chemicals,Truck,Canada,119860,0,2047,Domestically Produced Goods,Unknown,1,1/1/2022,NaN,01/01/2022,January
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52032,Import,55XX,Special classification provisions,Truck,Mexico,13411,1128,8,NaN,Not Containerized,3,1/1/2023,NaN,03/01/2023,March
52033,Import,60XX,Special classification provisions,Other,Canada,12995,86,450,NaN,Not Containerized,3,1/1/2023,NaN,03/01/2023,March
52034,Import,60XX,Special classification provisions,Other,Mexico,43000,7020,4505,NaN,Not Containerized,3,1/1/2023,NaN,03/01/2023,March
52035,Import,70XX,Imports only Temporary legislation Temporary m...,Other,Canada,246890556,0,0,NaN,Not Containerized,3,1/1/2023,NaN,03/01/2023,March


In [ ]:

#port_Trade_Data = port_Trade_Data.sort_values('Month&Year')
#port_Trade_Data = port_Trade_Data[port_Trade_Data['Month&Year'] == '02/01/2022']

In [ ]:
port_Trade_Data

# Creating new data sets based on certain parameters and downloading them.

In [62]:
#most_recent_month_trade_data=port_Trade_Data[(port_Trade_Data['Month&Year'] == '1/1/2023')]
#most_recent_month_trade_data.to_csv('January2023_Trade_Data.csv', index=False) #everything but only the most recent month

#san_Diego_County=port_Trade_Data
#san_Diego_County=san_Diego_County[(san_Diego_County['Port Code'] == '2507') | (san_Diego_County['Port Code'] == '2506')| (san_Diego_County['Port Code'] == '2505')| (san_Diego_County['Port Code'] == '2504')| (san_Diego_County['Port Code'] == '2503')| (san_Diego_County['Port Code'] == '2502')| (san_Diego_County['Port Code'] == '2501')]
#san_Diego_County.to_csv('SDBorderTrades01.csv', index=False) #Everything in SD County from all years
